In [1]:
import pandas as pd
import numpy as np 
import os

import torch

from preprocessing import drop_null_in_players
from graph import create_graph_from_dataset
from model_data import model_data, DiskDataset, check_data
from model import GNNRegression, train_nn





In [2]:
path_data_model = "../data/model_data/"
path_src_data = "../data/parquet/"
os.makedirs(path_data_model, exist_ok=True)

for name_game in os.listdir(path_src_data):

    if name_game not in os.listdir(path_data_model):

        data = pd.read_parquet(f"{path_src_data}{name_game}")
        data.sort_values(by=['quarter', 'game_clock'], ascending=[True, False], inplace=True)

        data[['awayscore', 'homescore']] = data[['awayscore', 'homescore']].ffill()
        data['shot_clock'] = data['shot_clock'].fillna(24)
        
        demo_data = data.iloc[0]    
        teams_list  = [demo_data['player2_team'],demo_data['player9_team']] 

        data_wo_empty_posessions = data[data['posession'].notna()]
        data_wo_null_values = drop_null_in_players(data=data_wo_empty_posessions)
        saved_graphs = create_graph_from_dataset(data=data_wo_null_values)
        y = np.where(data_wo_null_values['posession'] == 'home', data_wo_null_values['y_home'], data_wo_null_values['y_away'])

        data_list = model_data(saved_graphs=saved_graphs, y=y)
        check_data(data_list=data_list)

        save_dir = f'{path_data_model}{name_game}.pt'
        torch.save(data, save_dir)


/mnt/c/MDS/Q2/SDM/project/NBA-Player-Movements/analysis/graph.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.add_edge(row[0], row[1], weight=row[2])
/mnt/c/MDS/Q2/SDM/project/NBA-Player-Movements/analysis/graph.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.add_edge(row[0], row[1], weight=row[2])
/mnt/c/MDS/Q2/SDM/project/NBA-Player-Movements/analysis/graph.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  s

#### Create graph between players from a moment

#### Parse it to graph

#### Write to disk

In [ ]:
batch_size = 32
dataset = DiskDataset(directory=path_data_model)
loader = dataset.get_loader(batch_size=batch_size, shuffle=True)

In [ ]:
# Initialize the model, loss function, and optimizer
input_dim = 2 ## number of node features
hidden_dim = 16
output_dim = 1

model = GNNRegression(input_dim, hidden_dim, output_dim)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


train_nn(model=model, criterion=criterion, optimizer=optimizer, loader=loader)


Epoch 0, Loss: 0.924708117216159


KeyboardInterrupt: 